Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [46]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse

In [47]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [48]:
x = make_set_covering_problem(1000, 1000, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: False


## start solution

In [49]:
from random import randint
from functools import reduce
from queue import PriorityQueue
from copy import copy
from collections import deque

import numpy as np

PROBLEM_SIZE = 5000 # 100 1000 5000
NUM_SETS = PROBLEM_SIZE
DENSITY = .3 # .3 .7

SETS = make_set_covering_problem(PROBLEM_SIZE, NUM_SETS, DENSITY)

In [50]:
assert np.all(
    reduce(
        np.logical_or,
        [x for x in SETS.todense()],
        np.array([False for _ in range(PROBLEM_SIZE)]), #Starting from all False, make or of each set that is taken
    )
), "Probelm not solvable" # if using all sets there is no covering all

## Best solution (Third try)

In [51]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS.todense()[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

def get_candidate(state, sets_matrix):

    best = None
    best_contribution = 0

    for i, t in enumerate(state):
        if not t:
            contribution = -sum(sets_matrix[i])
            if contribution < best_contribution:
                best = i
                best_contribution = contribution
    
    return best, best_contribution

def modified_tweak(state, index):

    new_state = copy(state)
    new_state[index] = not new_state[index]

    return new_state

In [52]:
current_state = [False for _ in range(NUM_SETS)]
current_fitness = (0, 0)
not_covered_idx = [i for i in range(PROBLEM_SIZE)]

SETS_copy = SETS.copy()



for step in range(10_000):
    print(f'{step + 1}° step')

    SETS_copy_dense = SETS_copy.todense()

    t, v = get_candidate(current_state, SETS_copy_dense)

    not_covered_idx = np.where(SETS_copy_dense[t] == False)[0]

    SETS_copy = SETS_copy[:, not_covered_idx]

    current_state = modified_tweak(current_state, t)
    current_fitness = (current_fitness[0] - v, current_fitness[1] - 1)  ## idk if this is to be considered a fitness call or not (but i think yes)
    #current_fitness = fitness(current_state)

    print(current_fitness)
    print('-----------')

    if len(not_covered_idx) == 0: break

all_covered = np.all(
    reduce(
            np.logical_or,
            [SETS.todense()[i] for i, t in enumerate(current_state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)])
        )
)
print(f'all covered: {all_covered}')
print(f'number of fitness call: {step + 1}')
print(f'fitness: {current_fitness}')

1° step
(1613, -1)
-----------
2° step
(2723, -2)
-----------
3° step
(3480, -3)
-----------
4° step
(4012, -4)
-----------
5° step
(4357, -5)
-----------
6° step
(4593, -6)
-----------
7° step
(4752, -7)
-----------
8° step
(4851, -8)
-----------
9° step
(4919, -9)
-----------
10° step
(4959, -10)
-----------
11° step
(4983, -11)
-----------
12° step
(4996, -12)
-----------
13° step
(5000, -13)
-----------
all covered: True
number of fitness call: 13
fitness: (5000, -13)


## first try

In [53]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS.todense()[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

In [54]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS.todense()[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

In [55]:
current_state = [False for _ in range(NUM_SETS)]
current_fitness = fitness(current_state)

for step in range(10_000):
    new_state = tweak(current_state)
    new_fitness = fitness(new_state)
    if new_fitness >= current_fitness:
        current_state = new_state
        current_fitness = new_fitness
        print('------')
        print(f'new best at {step + 1}° iteration')
        print(current_fitness)
        if goal_check(current_state): break

all_covered = np.all(
    reduce(
            np.logical_or,
            [SETS.todense()[i] for i, t in enumerate(current_state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)])
        )
)
print(f'all covered: {all_covered}')
print(f'number of fitness call: {step + 2}')
print(f'fitness: {current_fitness}')


------
new best at 1° iteration
(1543, -1)
------
new best at 2° iteration
(2589, -2)
------
new best at 3° iteration
(3325, -3)
------
new best at 4° iteration
(3842, -4)
------
new best at 5° iteration
(4183, -5)
------
new best at 6° iteration
(4424, -6)
------
new best at 7° iteration
(4609, -7)
------
new best at 8° iteration
(4715, -8)
------
new best at 9° iteration
(4790, -9)
------
new best at 10° iteration
(4854, -10)
------
new best at 11° iteration
(4898, -11)
------
new best at 12° iteration
(4926, -12)
------
new best at 13° iteration
(4947, -13)
------
new best at 14° iteration
(4965, -14)
------
new best at 15° iteration
(4975, -15)
------
new best at 16° iteration
(4981, -16)
------
new best at 17° iteration
(4990, -17)
------
new best at 18° iteration
(4992, -18)
------
new best at 19° iteration
(4995, -19)
------
new best at 20° iteration
(4997, -20)
------
new best at 21° iteration
(4998, -21)
------
new best at 24° iteration
(5000, -22)
all covered: True
number of 

## second try (discarded)

In [56]:
#def modified_fitness(state):
#    
#    cost = sum(state)
#    covered = reduce(
#                np.logical_or,
#                [SETS.todense()[i] for i, t in enumerate(state) if t],
#                np.array([False for _ in range(PROBLEM_SIZE)]),
#            )
#    
#    return (np.sum(covered), -cost), covered
#
#
#def get_candidates(state, covered):
#
#    candidates = PriorityQueue()
#    not_covered = np.where(covered == False)
#    for i, t in enumerate(state):
#        if not t: candidates.put((-sum(SETS.todense()[i][not_covered]), i))
#
#    return candidates
#
#
#def modified_tweak(state, index):
#
#    new_state = copy(state)
#    new_state[index] = not new_state[index]
#
#    return new_state

In [57]:
#current_state = [False for _ in range(NUM_SETS)]
#current_fitness, covered = modified_fitness(current_state)
#
#for step in range(10_000):
#
#    candidates = get_candidates(current_state, covered)
#    v, t = candidates.get()
#
#    current_state = modified_tweak(current_state, t)
#    current_fitness = (current_fitness[0] - v, current_fitness[1] - 1)  ## idk if this is to be considered a fitness call or not (but i think yes)
#    covered = np.logical_or(covered, SETS.todense()[t])
#
#    print(f'{step + 1}° step')
#    print(current_fitness)
#    print('-----------')
#    
#    if goal_check(current_state): break
#
#all_covered = np.all(
#    reduce(
#            np.logical_or,
#            [SETS.todense()[i] for i, t in enumerate(current_state) if t],
#            np.array([False for _ in range(PROBLEM_SIZE)])
#        )
#)
#print(f'all covered: {all_covered}')
#print(f'number of fitness call: {step + 2}')
#print(f'fitness: {fitness(current_state)}')

## fourth try - Depth first (discarded)

In [58]:
frontier = deque()
state = [False for _ in range(NUM_SETS)]
frontier.append((state, [False for _ in range(PROBLEM_SIZE)]))

SETS_copy = SETS.todense()

counter = 0
current_state = frontier.pop()
while not np.all(current_state[1]):
    counter += 1
    for action in np.where(np.logical_not(current_state[0]))[0]:
        new_state = current_state[0].copy()
        new_state[action] = True
        new_covered = np.logical_or(current_state[1], SETS_copy[action])
        frontier.append((new_state, new_covered))
    current_state = frontier.pop()
    print(sum(current_state[1]))

all_covered = np.all(
    reduce(
            np.logical_or,
            [SETS_copy[i] for i, t in enumerate(current_state[0]) if t],
            np.array([False for _ in range(PROBLEM_SIZE)])
        )
)
print(f'all covered: {all_covered}')
print(f'number of fitness call: {counter + 1}')
print(f'fitness: {fitness(current_state[0])}')

1550
2572
3289
3813
4159
4414
4570
4698
4785
4847
4908
4938
4953
4966
4979
4988
4990
4992
4993
4996
4997
4999
4999
5000
all covered: True
number of fitness call: 25
fitness: (5000, -24)


## Fifth try - Greedy best first

In [59]:
def f(already_covered):
    missing_size = PROBLEM_SIZE - sum(already_covered)
    return missing_size

In [60]:
frontier = PriorityQueue()
state = [False for _ in range(NUM_SETS)]
frontier.put((f(state), (state, [False for _ in range(PROBLEM_SIZE)])))

SETS_copy = SETS.todense()

counter = 0
_, current_state = frontier.get()

while not np.all(current_state[1]):
    counter += 1
    for action in np.where(np.logical_not(current_state[0]))[0]:
        new_state = current_state[0].copy()
        new_state[action] = True
        new_covered = np.logical_or(current_state[1], SETS_copy[action])
        frontier.put((f(new_covered), (new_state, new_covered)))
    _, current_state = frontier.get()
    print(sum(current_state[1]))

all_covered = np.all(
    reduce(
            np.logical_or,
            [SETS_copy[i] for i, t in enumerate(current_state[0]) if t],
            np.array([False for _ in range(PROBLEM_SIZE)])
        )
)
print(f'all covered: {all_covered}')
print(f'number of fitness call: {counter + 1}')
print(f'fitness: {fitness(current_state[0])}')

1613
2723
3480
3998
4351
4585
4744
4851
4917
4958
4983
4995
5000
all covered: True
number of fitness call: 14
fitness: (5000, -13)
